In order to derive quantile forecasts from point forecasts, two approaches are typically employed.

## Rolling origin forecasting
Key Concepts:
Also known as time series cross-validation or walk-forward validation.

It allows for better evaluation of how the model performs in realistic, evolving scenarios.

Useful when future data points become available over time, and you want to continuously update forecasts.

How it works:
Start with an initial training window (e.g., time points 1 to 100).

Forecast the next period (e.g., point 101).

Roll the origin forward (e.g., now train on 1 to 101, forecast 102), or slide the window if using a fixed-size training set.

Repeat this process for the desired number of steps.


## Inventory control model
order-up-to policy = Vandeput,2020 inventory-optimization-models-and-simulations

Policy #2 – Periodic Review and Order Up-to Level (R,S)

> Specifically, an order-up-to policy is employed, as per Teunter and Sani (2009), where any out-of-stocks do not produce additional demand in the following periods (backorders), thus resulting in lost sales.


> From the book
> replenishment orders follow a specific schedule: every R periods we make an order to bring the net inventory level to S. When computing the safety stock in such a policy, we have to take into account the review period R, as well as the up-to level S

> Remember, the net inventory level is the available on-hand inventory and in-transit inventory, minus backorders, orders not yet shipped, etc.

In our example, we do not allow backorder, backorder=0.

net inventory = On-hand inventory + in-transit inventory - backorder - order not yet shiped

### how to calculate safety stock and up-to-level in this model

> inventory-optimization-models-and-simulations section 5.1.2

$$
ss = z_\alpha \sigma_\alpha \sqrt{R+L}
$$

$$
S = d_L+d_R+ss
$$

### code

```r
#calcualte safety stock  : https://abcsupplychain.com/safety-stock-formula-calculation/ = z*sigma_demand * sqrt(L)
SS<-function(SL, sd, L){
  # SL target service level
  if( any(SL <= 0 | SL >= 1) ) stop('SL not between 0 and 1')
  z<- qnorm(SL, 0, 1)
  rst <- z*sd*sqrt(L)
  rs <-   round(rst, 2)
  return(rs)
}

#ss is calculated under R+L time   ----- we can explain the logic of inventory control
ss<-SS(SL, sd(known_demand), (L+FT))


```



The simulation is initialized by assuming that the earliest stock level
is equal to zero and running the stock control process for 56 weeks.
Then, the first four weeks of the simulation are used as a burn-in period
and excluded from the analysis, resulting in a simulation period of 365
days. In addition, a full year prior to that is used in order to calculate
past forecast errors and enable the estimation of the safety stock. After
each review period, the forecast errors are updated, and so does the
safety stock. During the simulations, the holding stock (average daily
stock stored during the simulation period) and the lost sales (sum of
unfulfilled demand during the simulation period) are tracked for each
product. Moreover, the realized service level (probability of not having
a stock-out) and the fill rate (percentage of serviced demand over
effective demand) are computed.

## Theoretical Background of the Simulation

We use a $(R, S)$ model.

Suppose at time step $t$, the inventory reaches $S$, denoted by target inventory level $SS_t$. We need to determine the safety stock for the next $R+L$ period.

The constraint is "service level" equals $TSL$ (at least exactly). We use type 1 service level, i.e. probability of not stocking out in the cycle $R+L$.

> In other words, this is the probability that there is no stock-out between the time you receive a replenishment until the next replenishment arrives.

> This is the probability that the inventory available at the beginning of an order cycle (that is just after we received the order from the previous order cycle) will be greater than (or equal to) the demand during this cycle.

<!--
Suppose we receive the replenishment at time $T = t$. Then we are sure we make the order at $T = t-L$ (L lead time). Then we are sure, the time $T = t-L$ is when we do the inventory review. Then after $R$ (review period), we will do next review. i.e. at $T = t-L+R$, we do the next review, then the order for next review will arrive at $T= t-L+R+L = t+R$.

$I(t)$ is on-hand inventory when periods $t$ ends, in the setting, we need to calculate the probability at the beginning of order cycle, on-hand inventory (at this time demand at t is not sent out, we use $I(t-)$) larger than all demand during $t$ to $t+R$.

$T=t-L$, we do the review, on-hand inventory is $I(t-L)$.

The demand $(t-L, t-1)$ is
$$
\sum_{w=t-L+1}^{t-1} D_w
$$
-->





For each time step, we have demand $D_t$ comes. We have prediction for this value $\hat{D_t}$. So the total predicted demand during $R+L$ is 

$$
\sum_{w=t}^{t+R+L} \hat{D}_w
$$

The real (true) demand of this cycle is

$$
\sum_{w=t}^{t+R+L} \hat{D}_w
$$

We regard the real demands as independent random variables, and we regard the prediction as their expectation. We further assume a homoskedastic variance, i.e. homoskedastic error mentioned in paper
> However, it assumes homoskedastic errors, which can be a strong assumption in some cases, especially in retail where factors like seasonality and promotions influence not just average sales but also their variance.

That is
$$
\mathbb{E}(D_t) = \hat{D_t},\quad\quad \sigma^2 = Var(D_t)= \text{const},\,\, \forall t
$$

Note that we can use [Lyapunov CLT](https://en.wikipedia.org/wiki/Central_limit_theorem) to approx.

$$
\frac{1}{\sqrt{n}\sigma} \sum_{i=1}^{n}(D_i-\hat{D}_i) \xrightarrow{d} N(0,1)
$$
i.e.
$$
\sum_i D_i \approx N \left(\sum_i\hat{D}_i, n\sigma^2\right)
$$

In our case,
$$
\sum_{w=t}^{t+R+L} D_w \approx  N \left(\sum_{w=t}^{t+R+L}\hat{D}_w, (R+L)\sigma^2\right)
$$

Now we consider the constraint
$$
\begin{aligned}
\mathbb{P}(\text{not stocking out}) &= TSL \\
                                    &= \mathbb{P}( SS_t - [\text{total demand}] \geq 0) \\ 
                                    &= \mathbb{P}( SS_t - \sum_{w=t}^{t+R+L} D_w  \geq 0) \\ 
\end{aligned}
$$

Let $e=\sum D_w - \sum \hat{D}_w$, then $e \sim N(0,  (R+L)\sigma^2 )$, so
$$
\begin{aligned}
TSL = \mathbb{P} (SS_t - \sum \hat{D}_w  \geq e ) = \mathbb{P} \left(\frac{SS_t - \sum \hat{D}_w }{\sqrt{R+L} \sigma}  \geq \frac{e}{\sqrt{R+L} \sigma} \right)
\end{aligned}
$$

Then, we have
$$
SS_t = \Phi(TSL) \cdot \sqrt{R+L} \cdot \sigma +  \sum_{w=t}^{t+R+L} \hat{D}_w
$$

We define the first term as safety stock
$$
ss_t = \Phi(TSL) \cdot \sqrt{R+L} \cdot \sigma
$$
where $\sigma$ can be estimated from historical true/actual demand.

The above is what the code does.



## Meaning of variables in `RS` function

L: lead time  
SL: Target service level  
FT: review periods  
sent: actual decrease of inventory or sent  
orders: how much we order at review point    
s: on hand inventory level

## Data dim in `RS` function

